In [1]:
import pandas as pd
df=pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [2]:
cols = ['RowNumber','CustomerId','Surname']
df.drop(cols,inplace=True,axis=1)


In [3]:
from sklearn.preprocessing import OrdinalEncoder

object_cols = ["Geography", "Gender"]

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [16]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already loaded your dataset into X and y.
# If not, replace this with your data loading process.
# X: Features, y: Target labels

# Initialize the KFold cross-validator with 5 folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

y_pred_list = []
y_prob_list = []

# Loop through the folds
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create and train your classifier (e.g., RandomForestClassifier)
    classifier = RandomForestClassifier()
    classifier.fit(X_train, y_train)

    # Predict on the test set
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)  # Predict probabilities for each class

    y_pred_list.append(y_pred)
    y_prob_list.append(y_prob)

    # Calculate and print accuracy for each fold (optional)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Fold {len(y_pred_list)} - Accuracy: {accuracy:.4f}")

# Convert the lists to arrays
y_pred_array = np.concatenate(y_pred_list)
y_prob_array = np.concatenate(y_prob_list)

# Print the final y_pred and y_prob
print("Final y_pred:")
print(y_pred_array)

print("Final y_prob:")
print(y_prob_array)


KeyError: "None of [Int64Index([   1,    2,    4,    5,    6,    7,    9,   11,   13,   15,\n            ...\n            9987, 9988, 9990, 9991, 9992, 9994, 9995, 9996, 9997, 9999],\n           dtype='int64', length=8000)] are in the [columns]"

In [6]:
incorrect_predictions_df = result_df[result_df['y_pred'] != result_df['y']]
incorrect_prediction_indexes = incorrect_predictions_df.index
incorrect_prediction_features = X.iloc[incorrect_prediction_indexes]

# Add the feature columns to the incorrect_predictions_df
incorrect_predictions_df = pd.concat([incorrect_predictions_df, incorrect_prediction_features], axis=1)

In [7]:
def average_ranking_loss_with_penalty(y_true, rankings):
    sorted_indices = np.argsort(rankings)
    sorted_true = y_true[sorted_indices]
    sorted_ranking = rankings[sorted_indices]
    
    true_indices = np.where(sorted_true == 1)[0]
    false_indices = np.where(sorted_true == 0)[0]
    
    false_rankings = sorted_ranking[false_indices]
    higher_rankings = false_rankings > np.expand_dims(sorted_ranking[true_indices], axis=1)
    equal_rankings = false_rankings == np.expand_dims(sorted_ranking[true_indices], axis=1)
    
    PENNi_sum = np.sum(higher_rankings, axis=1) + 0.5 * np.sum(equal_rankings, axis=1)
    numerator_sum = np.sum(sorted_true[true_indices] * PENNi_sum)
    denominator_sum = np.sum(y_true)
    
    return numerator_sum / denominator_sum

In [8]:
y_true = np.array(result_df.y)
rankings = np.array(result_df.y_pred_prob)
average_ranking_loss_with_penalty(y_true,rankings)

2623.911143838979

In [9]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y, y_pred_prob)
print(f"AUC Score: {auc_score:.2f}")

AUC Score: 0.67


In [10]:
import pysubgroup as ps
#Numeric
target = ps.NumericTarget ('Exited')
searchspace = ps.create_selectors(df, ignore=['Exited'])
task = ps.SubgroupDiscoveryTask (
    df, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.StandardQFNumeric(1.2))
result = ps.BeamSearch().execute(task)
print(result.to_dataframe())

       quality                                subgroup  size_sg  size_dataset  \
0  2436.124301                                 Age>=46   2111.0       10000.0   
1  2057.798567                        NumOfProducts==1   5084.0       10000.0   
2  1966.067499  IsActiveMember==0 AND NumOfProducts==1   2521.0       10000.0   
3  1778.275024            Age>=46 AND NumOfProducts==1   1244.0       10000.0   
4  1715.606751                       IsActiveMember==0   4849.0       10000.0   

    mean_sg  mean_dataset    std_sg  std_dataset  median_sg  median_dataset  \
0  0.453340        0.2037  0.497818     0.402748        0.0             0.0   
1  0.277144        0.2037  0.447588     0.402748        0.0             0.0   
2  0.366521        0.2037  0.481854     0.402748        0.0             0.0   
3  0.547428        0.2037  0.497746     0.402748        1.0             0.0   
4  0.268509        0.2037  0.443184     0.402748        0.0             0.0   

   max_sg  max_dataset  min_sg  min_da

C:\Users\Nihar\anaconda3\lib\site-packages\pysubgroup\numeric_target.py:70: RuntimeWarning: invalid value encountered in double_scalars
  statistics['median_lift'] = statistics['median_sg'] / statistics['median_dataset']
C:\Users\Nihar\anaconda3\lib\site-packages\pysubgroup\numeric_target.py:70: RuntimeWarning: divide by zero encountered in double_scalars
  statistics['median_lift'] = statistics['median_sg'] / statistics['median_dataset']
C:\Users\Nihar\anaconda3\lib\site-packages\pysubgroup\utils.py:234: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(res, columns=headers, dtype=np.float64)


In [11]:
import pysubgroup as ps
#Numeric
target = ps.NumericTarget ('y')
searchspace = ps.create_selectors(incorrect_predictions_df, ignore=['y','y_pred','y_pred_prob'])
task = ps.SubgroupDiscoveryTask (
    incorrect_predictions_df, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=1, 
    qf=ps.StandardQFNumeric(1.25))
result = ps.BeamSearch().execute(task)
print(result.to_dataframe())

      quality           subgroup  size_sg  size_dataset   mean_sg  \
0  497.876455  IsActiveMember==0   1243.0        2102.0  0.982301   
1  161.710513       Age: [37:43[    429.0        2102.0  0.997669   
2  159.305146   CreditScore>=732    424.0        2102.0  0.997642   
3  152.333461             Age<37    400.0        2102.0  1.000000   
4  138.641493       Age: [43:48[    409.0        2102.0  0.990220   

   mean_dataset    std_sg  std_dataset  median_sg  median_dataset  max_sg  \
0      0.914843  0.131855     0.279115        1.0             1.0     1.0   
1      0.914843  0.048224     0.279115        1.0             1.0     1.0   
2      0.914843  0.048507     0.279115        1.0             1.0     1.0   
3      0.914843  0.000000     0.279115        1.0             1.0     1.0   
4      0.914843  0.098409     0.279115        1.0             1.0     1.0   

   max_dataset  min_sg  min_dataset  mean_lift  median_lift  
0          1.0     0.0          0.0   1.073737          1.0 

C:\Users\Nihar\anaconda3\lib\site-packages\pysubgroup\utils.py:234: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(res, columns=headers, dtype=np.float64)


In [12]:
incorrect_predictions_df

,y_pred,y,y_pred_prob,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,1,0.194537,619,0.0,0.0,42,2,0.00,1,1,1,101348.88
2,0,1,0.430664,502,0.0,0.0,42,8,159660.80,3,1,0,113931.57
5,0,1,0.244802,645,2.0,1.0,44,8,113755.78,2,1,0,149756.71
7,0,1,0.406900,376,1.0,0.0,29,4,115046.74,4,1,0,119346.88
16,0,1,0.430993,653,1.0,1.0,58,1,132602.88,1,1,0,5097.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,0,1,0.450735,498,1.0,1.0,42,3,152039.70,1,1,1,53445.17
9982,0,1,0.277949,655,1.0,0.0,46,7,137145.12,1,1,0,115146.40
9991,0,1,0.385429,597,0.0,0.0,53,4,88381.21,1,1,0,69384.71
9997,0,1,0.112706,709,0.0,0.0,36,7,0.00,1,0,1,42085.58


In [13]:
incorrect_predictions_df.loc[incorrect_predictions_df.Age>=55]

,y_pred,y,y_pred_prob,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
16,0,1,0.430993,653,1.0,1.0,58,1,132602.88,1,1,0,5097.67
42,1,0,0.540703,556,0.0,0.0,61,2,117419.35,1,1,1,94153.83
70,0,1,0.323874,738,1.0,1.0,58,2,133745.44,4,1,0,28373.86
104,0,1,0.315883,670,2.0,0.0,65,1,0.00,1,1,1,177655.68
127,0,1,0.444752,625,1.0,1.0,56,0,148507.24,1,1,0,46824.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9859,0,1,0.311026,678,1.0,1.0,55,4,129646.91,1,1,1,184125.10
9894,1,0,0.661091,521,0.0,0.0,77,6,0.00,2,1,1,49054.10
9895,0,1,0.343741,684,1.0,0.0,56,3,127585.98,3,1,1,80593.49
9910,0,1,0.390133,624,0.0,1.0,55,7,118793.60,1,1,1,95022.02


In [14]:
#Binary
target = ps.BinaryTarget ('y_pred', False)
searchspace = ps.create_selectors(incorrect_predictions_df, ignore=['y','y_pred','y_pred_prob'])
task = ps.SubgroupDiscoveryTask (
    incorrect_predictions_df, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=2, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
print(result.to_dataframe())

C:\Users\Nihar\anaconda3\lib\site-packages\pysubgroup\utils.py:234: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(res, columns=headers, dtype=np.float64)


    quality                                       subgroup  size_sg  \
0  0.039891                              IsActiveMember==0   1243.0   
1  0.029153         IsActiveMember==0 AND NumOfProducts==1    884.0   
2  0.028333             HasCrCard==1 AND IsActiveMember==0    899.0   
3  0.023277              Gender==0.0 AND IsActiveMember==0    692.0   
4  0.017645  Balance: [0.0:98792.40[ AND IsActiveMember==0    506.0   

   size_dataset  positives_sg  positives_dataset  size_complement  \
0        2102.0        1221.0             1923.0            859.0   
1        2102.0         870.0             1923.0           1218.0   
2        2102.0         882.0             1923.0           1203.0   
3        2102.0         682.0             1923.0           1410.0   
4        2102.0         500.0             1923.0           1596.0   

   relative_size_sg  relative_size_complement  coverage_sg  \
0          0.591342                  0.408658     0.634945   
1          0.420552               

In [15]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already loaded your dataset into X and y.
# If not, replace this with your data loading process.
# X: Features, y: Target labels

# Initialize the KFold cross-validator with 5 folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

y_pred_list = []
y_prob_list = []

# Loop through the folds
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create and train your classifier (e.g., RandomForestClassifier)
    classifier = RandomForestClassifier()
    classifier.fit(X_train, y_train)

    # Predict on the test set
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)  # Predict probabilities for each class

    y_pred_list.append(y_pred)
    y_prob_list.append(y_prob)

    # Calculate and print accuracy for each fold (optional)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Fold {len(y_pred_list)} - Accuracy: {accuracy:.4f}")

# Convert the lists to arrays
y_pred_array = np.concatenate(y_pred_list)
y_prob_array = np.concatenate(y_prob_list)

# Print the final y_pred and y_prob
print("Final y_pred:")
print(y_pred_array)

print("Final y_prob:")
print(y_prob_array)


KeyError: "None of [Int64Index([   1,    2,    4,    5,    6,    7,    9,   11,   13,   15,\n            ...\n            9987, 9988, 9990, 9991, 9992, 9994, 9995, 9996, 9997, 9999],\n           dtype='int64', length=8000)] are in the [columns]"